Performance Tuning Guide 是一组优化和最佳实践，可以加速 PyTorch 中深度学习模型的训练和推理。 提出的技术通常只需更改几行代码即可实现，并且可以应用于所有领域的各种深度学习模型。

# 一般优化

## 启用异步数据加载和扩充

torch.utils.data.DataLoader 在单独的工作子进程中支持异步数据加载和数据扩充。 DataLoader 的默认设置是 num_workers=0，这意味着数据加载是同步的，并且在主进程中完成。 因此，主训练过程必须等待数据可用才能继续执行。

设置 num_workers > 0 启用异步数据加载以及训练和数据加载之间的重叠。 num_workers 应该根据工作负载、CPU、GPU 和训练数据的位置进行调整。

DataLoader 接受 pin_memory 参数，默认为 False。 使用 GPU 时，最好设置 pin_memory=True，这会指示 DataLoader 使用固定内存并启用从主机到 GPU 的更快的异步内存复制。

## 禁用梯度计算以进行验证或推理

PyTorch 从所有涉及需要梯度的张量的操作中保存中间缓冲区。通常，验证或推理不需要梯度。 torch.no_grad() 上下文管理器可用于禁用指定代码块内的梯度计算，这可以加速执行并减少所需的内存量。 torch.no_grad() 也可以用作函数装饰器。

## 禁用卷积的偏差，直接跟随一个BN

torch.nn.Conv2d() 具有默认为 True 的偏差参数（对于 Conv1d 和 Conv3d 也是如此）。

如果 nn.Conv2d 层直接跟在 nn.BatchNorm2d 层之后，则不需要卷积中的偏差，而是使用 nn.Conv2d(..., bias=False, ....)。不需要偏差，因为在第一步 BatchNorm 减去平均值，这有效地抵消了偏差的影响。

这也适用于 1d 和 3d 卷积，只要 BatchNorm（或其他归一化层）在与卷积偏差相同的维度上归一化。

torchvision 提供的模型已经实现了这种优化。

## 使用 `parameter.grad = None` 而不是 `model.zero_grad()` 或 `optimizer.zero_grad()`

而不是调用：

In [ ]:
model.zero_grad()
# or
optimizer.zero_grad()

要将梯度归零，请改用以下方法：

In [ ]:
for param in model.parameters():
    param.grad = None

第二个代码片段没有将每个单独参数的内存归零，后续的反向传递也使用赋值而不是加法来存储梯度，这减少了内存操作的数量。

将梯度设置为无与将其设置为零的数值行为略有不同，有关更多详细信息，请参阅文档。

或者，从 PyTorch 1.7 开始，调用 `model` 或 `optimizer.zero_grad(set_to_none=True)`。

## 融合逐点操作

逐点运算（按元素加法、乘法、数学函数 - sin()、cos()、sigmoid() 等）可以融合到单个内核中，以分摊内存访问时间和内核启动时间。

PyTorch JIT 可以自动融合内核，尽管编译器中可能还没有实现其他融合机会，并且并非所有设备类型都得到同等支持。

Pointwise 操作受内存限制，对于每个操作 PyTorch 都会启动一个单独的内核。 每个内核从内存加载数据，执行计算（这一步通常很便宜）并将结果存储回内存。

Fused operator 仅为多个融合 pointwise ops 启动一个内核，并且仅将数据加载/存储一次到内存。 这使得 JIT 对于激活函数、优化器、自定义 RNN 单元等非常有用。

在最简单的情况下，可以通过将 torch.jit.script 装饰器应用于函数定义来启用融合，例如：

In [ ]:
@torch.jit.script
def fused_gelu(x):
    return x * 0.5 * (1.0 + torch.erf(x / 1.41421))

有关更高级的用例，请参阅 TorchScript 文档。

## 为计算机视觉模型启用 channels_last 内存格式

PyTorch 1.5 引入了对卷积网络的 channels_last 内存格式的支持。 这种格式旨在与 AMP 结合使用，以进一步加速具有 Tensor Cores 的卷积神经网络。

对 channels_last 的支持是实验性的，但预计它适用于标准计算机视觉模型（例如 ResNet-50、SSD）。 要将模型转换为 channels_last 格式，请遵循 Channels Last Memory Format Tutorial。 本教程包括一个关于转换现有模型的部分。

## 检查点中间缓冲区

缓冲区检查点是一种减轻模型训练内存容量负担的技术。 它不是存储所有层的输入来计算反向传播中的上游梯度，而是存储几个层的输入，其他层在反向传播期间重新计算。 减少的内存需求可以增加批次大小，从而提高利用率。

应仔细选择检查点目标。 最好不要存储具有小的重新计算成本的大层输出。 示例目标层是激活函数（例如 ReLU、Sigmoid、Tanh）、上/下采样和具有小累积深度的矩阵向量操作。

PyTorch 支持原生的 torch.utils.checkpoint API 来自动执行检查点和重新计算。

## 禁用调试 API

许多 PyTorch API 用于调试，应该在常规训练运行时禁用：
* 异常检测：torch.autograd.detect_anomaly 或 torch.autograd.set_detect_anomaly(True)
* 分析器相关：torch.autograd.profiler.emit_nvtx、torch.autograd.profiler.profile
* autograd gradcheck：torch.autograd.gradcheck 或 torch.autograd.gradgradcheck

# CPU 特定的优化

## 利用非统一内存访问 (NUMA) 控件

NUMA 或非均匀内存访问是一种用于数据中心机器的内存布局设计，旨在利用具有多个内存控制器和块的多插槽机器中的内存局部性。 一般来说，所有深度学习工作负载、训练或推理，无需跨 NUMA 节点访问硬件资源即可获得更好的性能。 因此，推理可以使用多个实例运行，每个实例运行在一个套接字上，以提高吞吐量。 对于单节点的训练任务，建议分布式训练，使每个训练过程都运行在一个socket上。

在一般情况下，以下命令仅在第 N 个节点上的内核上执行 PyTorch 脚本，并避免跨套接字内存访问以减少内存访问开销。

In [ ]:
# numactl --cpunodebind=N --membind=N python <pytorch_script>

## 使用 OpenMP

OpenMP 用于为并行计算任务带来更好的性能。 OMP_NUM_THREADS 是可用于加速计算的最简单的开关。 它确定用于 OpenMP 计算的线程数。 CPU 亲和性设置控制工作负载如何分布在多个内核上。 它会影响通信开销、缓存行无效开销或页面抖动，因此正确设置 CPU 亲和性会带来性能优势。 GOMP_CPU_AFFINITY 或 KMP_AFFINITY 决定如何将 OpenMP* 线程绑定到物理处理单元。 详细信息可以在这里找到。

使用以下命令，PyTorch 在 N 个 OpenMP 线程上运行任务。

In [ ]:
# export OMP_NUM_THREADS=N

通常，以下环境变量用于设置与 GNU OpenMP 实现的 CPU 亲和性。 OMP_PROC_BIND 指定线程是否可以在处理器之间移动。 将其设置为 CLOSE 可使 OpenMP 线程靠近连续位置分区中的主线程。 OMP_SCHEDULE 确定如何调度 OpenMP 线程。 GOMP_CPU_AFFINITY 将线程绑定到特定的 CPU。

In [ ]:
# export OMP_SCHEDULE=STATIC
# export OMP_PROC_BIND=CLOSE
# export GOMP_CPU_AFFINITY="N-M"

## 英特尔 OpenMP 运行时库 (libiomp)

默认情况下，PyTorch 使用 GNU OpenMP (GNU libgomp) 进行并行计算。 在英特尔平台上，英特尔 OpenMP 运行时库 (libiomp) 提供 OpenMP API 规范支持。 与 libgomp 相比，它有时会带来更多的性能优势。 利用环境变量 LD_PRELOAD 可以将 OpenMP 库切换到 libiomp：

In [ ]:
# export LD_PRELOAD=<path>/libiomp5.so:$LD_PRELOAD

与 GNU OpenMP 中的 CPU 关联设置类似，libiomp 中提供了环境变量来控制 CPU 关联设置。 KMP_AFFINITY 将 OpenMP 线程绑定到物理处理单元。 KMP_BLOCKTIME 设置线程在完成并行区域的执行后，在休眠之前应等待的时间（以毫秒为单位）。 在大多数情况下，将 KMP_BLOCKTIME 设置为 1 或 0 会产生良好的性能。 以下命令显示了英特尔 OpenMP 运行时库的常见设置。

In [ ]:
# export KMP_AFFINITY=granularity=fine,compact,1,0
# export KMP_BLOCKTIME=1

## 切换内存分配器

对于深度学习工作负载，与默认的 malloc 函数相比，Jemalloc 或 TCMalloc 可以通过尽可能多地重用内存来获得更好的性能。 Jemalloc 是一个通用的 malloc 实现，它强调避免碎片和可扩展的并发支持。 TCMalloc 还提供了一些优化来加速程序执行。 其中之一是将内存保存在缓存中以加快对常用对象的访问。 如果以后重新分配此类内存，即使在释放后保留此类缓存也有助于避免代价高昂的系统调用。 使用环境变量 LD_PRELOAD 来利用其中之一。

In [ ]:
# export LD_PRELOAD=<jemalloc.so/tcmalloc.so>:$LD_PRELOAD

## 使用 PyTorch DistributedDataParallel(DDP) 功能在 CPU 上训练模型

对于小规模模型或内存受限模型，例如 DLRM，在 CPU 上训练也是一个不错的选择。 在具有多个插槽的机器上，分布式训练带来了高效的硬件资源使用，以加速训练过程。 Torch-ccl 与 Intel(R) oneCCL（集体通信库）一起优化，用于实现 allreduce、allgather、alltoall 等集体的高效分布式深度学习训练，实现 PyTorch C10D ProcessGroup API，并且可以作为外部 ProcessGroup 动态加载。 在 PyTorch DDP 模块中实施优化后，torhc-ccl 可加速通信操作。 除了对通信内核进行优化之外，torch-ccl 还具有同步计算通信功能。

# GPU 特定的优化

## 启用 cuDNN 自动调谐器

NVIDIA cuDNN 支持许多算法来计算卷积。 Autotuner 运行一个简短的基准测试，并为给定的输入大小选择在给定硬件上具有最佳性能的内核。

对于卷积网络（目前不支持的其他类型），在启动训练循环之前通过设置启用 cuDNN 自动调谐器：

In [ ]:
torch.backends.cudnn.benchmark = True

* 自动调谐器的决定可能是不确定的； 可以为不同的运行选择不同的算法。 有关更多详细信息，请参阅 PyTorch：再现性
* 在一些罕见的情况下，例如输入大小高度可变的情况下，最好在禁用自动调谐器的情况下运行卷积网络，以避免与每个输入大小的算法选择相关的开销。

## 避免不必要的 CPU-GPU 同步

避免不必要的同步，尽可能让CPU跑在加速器之前，保证加速器工作队列中包含很多操作。

如果可能，请避免需要同步的操作，例如：
* 打印（cuda_tensor）
* cuda_tensor.item()
* 内存副本：tensor.cuda()、cuda_tensor.cpu() 和等效的 tensor.to(device) 调用
* cuda_tensor.nonzero()
* python控制流取决于对cuda张量执行的操作结果，例如 如果 (cuda_tensor != 0).all()

## 直接在目标设备上创建张量

不是调用 torch.rand(size).cuda() 来生成随机张量，而是直接在目标设备上生成输出：torch.rand(size, device=torch.device('cuda'))。

这适用于所有创建新张量并接受设备参数的函数：torch.rand()、torch.zeros()、torch.full() 等。

## 使用混合精度和 AMP

混合精度利用 Tensor Cores 并在 Volta 和更新的 GPU 架构上提供高达 3 倍的整体加速。 要使用 Tensor Cores AMP 应该被启用并且矩阵/张量维度应该满足调用使用 Tensor Cores 的内核的要求。

要使用张量核心：
* 将大小设置为 8 的倍数（映射到张量核心的维度）
    * 有关特定于层类型的更多详细信息和指南，请参阅深度学习性能文档
    * 如果层大小来自其他参数而不是固定的，它仍然可以显式填充，例如 NLP 模型中的词汇量
* 启用 AMP
    * 混合精度训练和 AMP 简介：视频、幻灯片
    * 原生 PyTorch AMP 从 PyTorch 1.6 开始可用：文档、示例、教程

## 在可变输入长度的情况下预分配内存

用于语音识别或 NLP 的模型通常在具有可变序列长度的输入张量上进行训练。可变长度对于 PyTorch 缓存分配器来说可能是有问题的，并可能导致性能降低或意外的内存不足错误。如果一个序列长度较短的批次之后是另一个序列长度较长的批次，则 PyTorch 将被迫从前一次迭代中释放中间缓冲区并重新分配新缓冲区。此过程非常耗时，并且会导致缓存分配器中出现碎片，从而可能导致内存不足错误。

一个典型的解决方案是实现预分配。它由以下步骤组成：
1. 生成具有最大序列长度（对应于训练数据集中的最大长度或某个预定义阈值）的（通常是随机的）批次输入
1. 使用生成的批处理执行前向和后向传递，不执行优化器或学习率调度器，此步骤预先分配最大大小的缓冲区，可在后续训练迭代中重复使用
1. 将梯度归零
1. 进行常规训练

# 分布式优化

## 使用高效的数据并行后端

PyTorch 有两种实现数据并行训练的方法：
* torch.nn.DataParallel
* torch.nn.parallel.DistributedDataParallel

DistributedDataParallel 为多 GPU 提供了更好的性能和扩展。 有关更多信息，请参阅 PyTorch 文档中 CUDA 最佳实践的相关部分。

## 如果使用 DistributedDataParallel 和梯度累积进行训练，则跳过不必要的 all-reduce

默认情况下，torch.nn.parallel.DistributedDataParallel 在每次向后传递后执行梯度 all-reduce 以计算参与训练的所有工人的平均梯度。 如果训练在 N 步上使用梯度累积，那么在每个训练步骤之后都不需要 all-reduce，只需要在最后一次调用 backback 之后，就在优化器执行之前执行 all-reduce。

DistributedDataParallel 提供了 no_sync() 上下文管理器，它为特定的迭代禁用梯度 all-reduce。 no_sync() 应该应用于梯度累积的前 N-1 次迭代，最后一次迭代应该遵循默认执行并执行所需的梯度 all-reduce。

## 如果使用 DistributedDataParallel(find_unused_pa​​rameters=True)，则在构造函数和执行期间匹配层的顺序

torch.nn.parallel.DistributedDataParallel with find_unused_pa​​rameters=True 使用来自模型构造函数的层和参数的顺序为 DistributedDataParallel 梯度 all-reduce 构建桶。 DistributedDataParallel 将 all-reduce 与向后传递重叠。仅当给定存储桶中参数的所有梯度都可用时，才会异步触发特定存储桶的 All-reduce。

为了最大化重叠量，模型构造函数中的顺序应该与执行期间的顺序大致匹配。如果顺序不匹配，则整个bucket的all-reduce等待最后一个到达的梯度，这可能会减少backward pass和all-reduce之间的重叠，all-reduce可能最终被暴露，这减慢训练。

具有 find_unused_pa​​rameters=False（默认设置）的 DistributedDataParallel 依赖于基于向后传递期间遇到的操作顺序的自动存储桶形成。使用 find_unused_pa​​rameters=False 不需要重新排序层或参数以实现最佳性能。

## 在分布式环境中负载平衡工作负载

负载不平衡通常可能发生在处理顺序数据（语音识别、翻译、语言模型等）的模型中。 如果一个设备收到一批序列长度大于其余设备序列长度的数据，则所有设备都等待最后完成的工作程序。 向后传递在具有 DistributedDataParallel 后端的分布式设置中用作隐式同步点。

有多种方法可以解决负载均衡问题。 核心思想是在每个全局批次中尽可能均匀地将工作量分配给所有工人。 例如，Transformer 通过形成具有近似恒定数量的令牌（以及批次中可变数量的序列）的批次来解决不平衡，其他模型通过对具有相似序列长度的样本进行分桶甚至通过按序列长度对数据集进行排序来解决不平衡。